In [ ]:
import mysql.connector
import pandas as pd
import matplotlib.pyplot as plt
import pickle as pkl
from sklearn.preprocessing import MinMaxScaler

# --- Database connection details ---
DB_HOST = <YOUR DB HOST>
DB_USER = <YOUR DB USER>
DB_PASSWORD = <YOUR DB PASSWORD>
DB_NAME = <YOUR DB NAME>

In [ ]:
# --- Connect to the database ---
db_connection = mysql.connector.connect(
    host=DB_HOST,
    user=DB_USER,
    password=DB_PASSWORD,
    database=DB_NAME,
    charset="utf8mb4",
    collation="utf8mb4_unicode_ci"
)

# Create a cursor object
cursor = db_connection.cursor()

In [ ]:
# get tweet ids
query = "SELECT id FROM tweets"
cursor.execute(query)
results = cursor.fetchall()
tweet_ids = [result[0] for result in results]
print(f'Retrieved {len(tweet_ids)} tweet ids.')

In [ ]:
# get notes
placeholders = ", ".join(["%s"] * len(tweet_ids))
query = f"""SELECT 
    noteId,
    tweetId,
    classification,
    believable,
    harmful,
    misleadingOther,
    misleadingFactualError,
    misleadingManipulatedMedia,
    misleadingOutdatedInformation,
    misleadingMissingImportantContext,
    misleadingUnverifiedClaimAsFact,
    misleadingSatire,
    notMisleadingOther,
    notMisleadingFactuallyCorrect,
    notMisleadingOutdatedButNotWhenWritten,
    notMisleadingClearlySatire,
    notMisleadingPersonalOpinion
FROM notes WHERE tweetId IN ({placeholders})"""
cursor.execute(query, tweet_ids)
results = cursor.fetchall()
full_notes = pd.DataFrame(results, columns=[
    'noteId', 
    'tweetId', 
    'classification', 
    'believable', 
    'harmful', 
    'misleadingOther', 
    'misleadingFactualError', 
    'misleadingManipulatedMedia', 
    'misleadingOutdatedInformation', 
    'misleadingMissingImportantContext', 
    'misleadingUnverifiedClaimAsFact', 
    'misleadingSatire', 
    'notMisleadingOther', 
    'notMisleadingFactuallyCorrect', 
    'notMisleadingOutdatedButNotWhenWritten', 
    'notMisleadingClearlySatire', 
    'notMisleadingPersonalOpinion'
])
note_ids = [str(id) for id in list(full_notes['noteId'].unique())]
print(f'Retrieved {len(full_notes)} notes.')

In [ ]:
# get ratings for the notes
placeholders = ", ".join(["%s"] * len(note_ids))
query = f"""SELECT 
    noteId,
    raterParticipantId,
    helpfulnessLevel,
    ratedOnTweetId
FROM note_ratings WHERE noteId IN ({placeholders})"""
cursor.execute(query, note_ids)
results = cursor.fetchall()
full_ratings = pd.DataFrame(results, columns=[
    'noteId', 
    'raterParticipantId', 
    'helpfulnessLevel', 
    'ratedOnTweetId'
])
print(f'Retrieved {len(full_ratings)} ratings.')

In [ ]:
# get full tweets
query = f"""
SELECT 
    id,
    text,
    created_at,
    author_id,
    retweet_count,
    reply_count,
    like_count,
    quote_count,
    bookmark_count,
    impression_count,
    interaction_score,
    action_count
FROM tweets
"""
cursor.execute(query)
results = cursor.fetchall()
full_tweets = pd.DataFrame(results, columns=[
    'id', 
    'text', 
    'created_at', 
    'author_id', 
    'retweet_count', 
    'reply_count', 
    'like_count', 
    'quote_count', 
    'bookmark_count', 
    'impression_count',
    'interaction_score',
    'action_count'
])
print(f'Retrieved {len(full_tweets)} tweets.')

In [ ]:
# calculate interaction scores and externalities


# get ratings score for the notes
ratings_map = {
    'HELPFUL': 1,
    'SOMEWHAT_HELPFUL': 0,
    'NOT_HELPFUL': -1
}

full_ratings['rating_score'] = full_ratings['helpfulnessLevel'].map(ratings_map)
note_ratings = full_ratings[['noteId', 'rating_score']].groupby('noteId').mean().reset_index()
# add note_ratings to full_notes
full_notes = full_notes.merge(note_ratings, on='noteId', how='left')

# get externality score for the notes
full_notes.loc[full_notes['misleadingManipulatedMedia'] == 1, 'misleadingManipulatedMedia'] = -3
full_notes.loc[full_notes['misleadingMissingImportantContext'] == 1, 'misleadingMissingImportantContext'] = -3

full_notes.loc[full_notes['misleadingUnverifiedClaimAsFact'] == 1, 'misleadingUnverifiedClaimAsFact'] = -2
full_notes.loc[full_notes['misleadingFactualError'] == 1, 'misleadingFactualError'] = -2

full_notes.loc[full_notes['misleadingOutdatedInformation'] == 1, 'misleadingOutdatedInformation'] = -1
full_notes.loc[full_notes['misleadingSatire'] == 1, 'misleadingSatire'] = -1
full_notes.loc[full_notes['misleadingOther'] == 1, 'misleadingOther'] = -1


full_notes.loc[full_notes['notMisleadingOutdatedButNotWhenWritten'] == 1, 'notMisleadingOutdatedButNotWhenWritten'] = 1

full_notes.loc[full_notes['notMisleadingPersonalOpinion'] == 1, 'notMisleadingPersonalOpinion'] = 2
full_notes.loc[full_notes['notMisleadingClearlySatire'] == 1, 'notMisleadingClearlySatire'] = 2
full_notes.loc[full_notes['notMisleadingOther'] == 1, 'notMisleadingOther'] = 2

full_notes.loc[full_notes['notMisleadingFactuallyCorrect'] == 1, 'notMisleadingFactuallyCorrect'] = 3

full_notes['type_min'] = full_notes[
    ['misleadingManipulatedMedia',
     'misleadingMissingImportantContext',
     'misleadingUnverifiedClaimAsFact',
     'misleadingFactualError',
     'misleadingOutdatedInformation',
     'misleadingSatire',
     'misleadingOther',
     'notMisleadingOutdatedButNotWhenWritten',
     'notMisleadingPersonalOpinion',
     'notMisleadingClearlySatire',
     'notMisleadingOther',
     'notMisleadingFactuallyCorrect']].min(axis=1)
full_notes['externality'] = full_notes['type_min'] - full_notes['rating_score']

In [ ]:
# add externality scores to tweets
notes_score_by_tweet = full_notes[['tweetId', 'externality']].groupby('tweetId').mean().reset_index()
full_tweets = full_tweets.merge(notes_score_by_tweet, left_on='id', right_on='tweetId', how='left')

# filter out tweets with no impressions
full_tweets = full_tweets[full_tweets['impression_count'] > 0]

# add action count per 1000 impressions
full_tweets['action_count_per_1000_impressions'] = full_tweets['action_count'] / (full_tweets['impression_count'] / 1000)

# remove outlier with massive action count 
# Alina Habba tweetid: 1819110904216084953
# Drug free world tweetid: 1847181579103490414
full_tweets = full_tweets[full_tweets['action_count_per_1000_impressions'] < 150]

print(full_tweets[['interaction_score','externality', 'action_count']].describe())

In [ ]:
# scale scores
# rescale externality to be between -1 and 1 while preserving the sign
externality = full_tweets['externality']
min_val, max_val = externality.min(), externality.max()
full_tweets['externality_scaled'] = -1 + (externality-min_val) * (1-(-1)) / (max_val-min_val)

# rescale interaction score to be between 0 and 1
interaction_score = full_tweets['interaction_score']
min_val, max_val = interaction_score.min(), interaction_score.max()
full_tweets['interaction_score_scaled'] = (interaction_score-min_val) * (1-0) / (max_val-min_val)

# rescale action count to be between 0 and 1
action_count = full_tweets['action_count']
min_val, max_val = action_count.min(), action_count.max()
full_tweets['action_count_scaled'] = (action_count-min_val) * (1-0) / (max_val-min_val)

# get externality in dollars
# convert greatest externality magnitude to 1 and then multiply by 1.53 dollars per 1000 impressions (CPM)
# from the goldstein 2014 paper, the cost of bad ads was $1.53 CPM
externality = full_tweets['externality']
greatest_magnitude = abs(externality.min())
full_tweets['externality_cpm'] = (externality/greatest_magnitude) * 1.53

print(full_tweets[['interaction_score_scaled','externality_scaled', 'action_count_scaled']].describe())

In [ ]:
# plot scaled interactions vs externality

fig, ax = plt.subplots()

ax.scatter(full_tweets['externality_cpm'], full_tweets['action_count_per_1000_impressions'], alpha=0.5)
ax.set_xlabel('Externality')
ax.set_ylabel('Action Count')
ax.set_title('Tweet Action Count vs Externality')

# Move the left spine (which is normally at x=min) to x=0
ax.spines['left'].set_position('zero')
# Hide the right spine
ax.spines['right'].set_color('none')

# If you also want the x-axis drawn at y=0, do likewise:
ax.spines['bottom'].set_position('zero')
ax.spines['top'].set_color('none')

# Set tick positions so they show on the moved spines
ax.xaxis.set_ticks_position('bottom')
ax.yaxis.set_ticks_position('left')

plt.show()

In [ ]:
# plot histograms of externality cpm and action counts per 1000 impressions
fig, ax = plt.subplots(2, 1, figsize=(10, 10))
ax[0].hist(full_tweets['externality_cpm'], bins=100)
ax[0].set_xlabel('Externality')
ax[0].set_ylabel('Count')
ax[0].set_title('Distribution of Externality')
ax[1].hist(full_tweets['action_count_per_1000_impressions'], bins=100, log=True )
ax[1].set_xlabel('Action Count')
ax[1].set_ylabel('Count')
ax[1].set_title('Distribution of Action Count')
plt.show()

In [ ]:
print(full_tweets[['impression_count']].describe())

In [ ]:
# --- Clean up ---
full_tweets.to_csv('full_tweets.csv', index=False)
cursor.close()
db_connection.close()